# 04 Uudiste meelsus (tonaalsus, polaarsus)
Leksikonipõhine  ja sõnapõhine lähenemine

Inspiratsioon ja algallikad:
1. Väärinforiskid Eesti Meediaturul
GDI(The Global Disinformation Index - Globaalne Väärinformatsiooni Indeks) hindab uudistesaitide võimalikku kalduvust väärinfo vahendamisele.
"Artiklite tonaalsus on hea indikaator, mille abil ennustada uudisportaali teisi
väärinforiskidega seotud tugevuste ja nõrkuste seost teiste muutujatega.
Siin on oluline välja tuua tähelepanek, et artikli tonaalsuse ja väärinfo esinemise
tõenäosuse vahel on tugev seos."
https://disinformationindex.org/wp-content/uploads/2020/10/GDI-RiskRatingsReport_Estonian.pdf
2. IDENTIFYING POLARITY IN DIFFERENT TEXT TYPES
http://www.folklore.ee/folklore/vol64/polarity.pdf
3. Meelestatuse analüüs
http://samm.ut.ee/meelestatuse-anal%C3%BC%C3%BCs
4. Quantifying Mental Health Signals in Twitter
https://www.aclweb.org/anthology/W14-3207.pdf
5. Emotsioonidetektor
http://peeter.eki.ee:5000/valence
6.Emotsioonid – Kõnetehnoloogia olevik ja tulevik Hille Pajupuu
https://www.academia.edu/7169398/Emotsioonid_k%C3%B5netehnoloogia_olevik_ja_tulevik
7.Skriptimisvahendid teksti lokaliseerimiseks ja asukohapõhiseks analüüsiks Eesti asulate meediakuvandi tuvastamise näitel
Ott Koik (2.2.5 Meelsusanalüüs)
https://dspace.ut.ee/bitstream/handle/10062/60459/Koik_Ott.pdf?sequence=1&isAllowed=y
8. This is a program classifying an Estonian text as positive, neutral or negative.
https://github.com/EKT1/valence
9.If you use the corpora valence/korpus.csv, valence/sqnad.csv in your work, please cite the following paper:

Pajupuu, Hille; Altrov, Rene; Pajupuu, Jaan (2016). Identifying polarity in different text types. Folklore. Electronic Journal of Folklore, 64, 25−42. DOI PDF
10.THE STRUCTURE OF ESTONIAN CONCEPTS OF EMOTION: A SELF-ORGANIZATIONAL APPROACH
https://www.researchgate.net/profile/Toomas_Kirt/publication/228689514_The_structure_of_estonian_concepts_of_emotion_A_self-organizational_approach/links/02e7e5164577f55fcc000000/The-structure-of-estonian-concepts-of-emotion-A-self-organizational-approach.pdf?origin=publication_list

Mida tehti?

1) Kasutada Eesti Keele Instituudi emotsioonileksikoni (estonian-emotion-lexicon.csv), mis sisaldab 39100 eestikeelset sõna koos skooriga. Skoor võib olla kas negatiivne (-1, -8) või positiivne (+1);

2) Kasutada EstNLTK teeki sõnaliikide tuvastamiseks

    a) esimesele isikule viitavate verbivormide leidmine ('gem','ks','ksime', 'ksin', 'me', 'n', 'neg gem', 'neg ks', 'neg me', 'neg nud', 'neg nuks', 'neg o', 'neg vat', 'nuks','nuksime', 'nuksin', 'nuvat', 'sime', 'sin', 'vat')
    
    b) esimesele isikule viitavate asesõnade leidmine ('mina', 'ma', 'meie', 'me')

Miks tehti? 

Algallikatele toetudes arvati, et negatiivsus ja esimese isiku kasutamine võivad olla uudise meelsuse näitajaks. (Eestikeelse emotsioonileksikoni kasutamise kohta on tehtud teadustöid, millele tugineti. Esimese isiku kasutamise kohta eestikeelses teaduskirjanduses viiteid ei leitud. Esimese isiku kasutamist näitajana peab seetõttu veel kaaluma.)

Kuidas?

Eesmärgiks leida uudisartiklile meelsuse (tonaalsuse, polaarsuse ) näitaja, mis arvestab artiklis kasutatud sõnade negatiivsust ja esimese isiku kasutust (verbid, asesõnad). 

Mis leiti?

Iga uudisartikli kohta leiti:

1) uudise pikkus sõnades

2) negatiivsete sõnade arv;

3) esimeses isikus kasutatud sõnade arv;

4) uudise skoor = negatiivsete sõnade arv + esimeses isikus kasutatud sõnade arv;

5) uudise kordaja = uudise skoor/uudise pikkus sõnades*;

6) positiivsete sõnade arv;

Kogu korpuse kohta leiti:
eelmistes punktides toodu, lisaks artiklites kasutatud negatiivsete ja positiivsete sõnade list.

*Positiivsete sõnade arv ei kajastu kordajas.

Väljastatakse skooride tabel, mille veergudeks on:

sõnade arv uudises

esimeses isikus sõnade arv

negatiivsete sõnade arv

uudise kordaja

positiivsete sõnade arv

väljaande nimi

uudise pealkiri

In [1]:
import csv
import pandas as pd
from estnltk import Text
from tqdm import tqdm

In [2]:
asesõnad = ['mina', 'ma', 'meie', 'me']
verbid = ['gem','ks','ksime', 'ksin', 'me', 'n', 'neg gem', 'neg ks', 'neg me', 'neg nud', 'neg nuks', 'neg o', 'neg vat', 'nuks'
         'nuksime', 'nuksin', 'nuvat', 'sime', 'sin', 'vat']
leksikon = {}
with open('estonian-emotion-lexicon.csv') as csv_file:
    reader = csv.reader(csv_file, delimiter=';')
    header = next(reader)
    for row in reader:
        leksikon[row[0]]= row[1]

In [3]:
def uudise_skoor(uudised): # sisend DataFrame
    esimese_isiku_kasutus = 0
    negatiivsus = 0
    positiivsus = 0
    negatiivsed_sonad_list = []
    positiivsed_sonad_list = []
    uudiste_pikkus_sõnades = 0
    uudisdict= {}
    pikkus =[]
    esimene_isik = []
    neg = []
    kordaja = []
    pos =[]
    print("Analüüsitakse", len(uudised.text),"uudist.")
    
    for i in tqdm(range (0, len(uudised))):
        rida = uudised.iloc[i]
        #print(i, rida[0])
        #print(rida[1])
        #print(len(rida[1]))
        text = Text(rida[1]).tag_layer()#.morph_analysis['lemma','partofspeech']

    #for i, uudis in tqdm(enumerate(uudised.text)):
    
        #text = Text(uudis).tag_layer()
        uudise_pikkus_sõnades = len(text.words.text)
        if uudise_pikkus_sõnades !=0:
            uudiste_pikkus_sõnades += uudise_pikkus_sõnades

            a_sõnade_arv = 0
            v_sõnade_arv = 0
            esimene_isik_arv = 0
            negatiivsete_arv = 0
            positiivsete_arv = 0

            morf = text.morph_analysis
            for sõna in morf:

                if sõna.text in leksikon:

                    if int(leksikon[sõna.text]) < 0:
                        negatiivsed_sonad_list.append(sõna.text)
                        negatiivsete_arv -= int(leksikon[sõna.text])

                    if int(leksikon[sõna.text]) > 0:
                        positiivsed_sonad_list.append(sõna.text)
                        positiivsete_arv += int(leksikon[sõna.text])

                if sõna.partofspeech[0]  == 'V': #verb
                    if sõna.form[0] in verbid:
                        v_sõnade_arv += 1
                if sõna.text in asesõnad:
                    a_sõnade_arv += 1
            esimene_isik_arv = a_sõnade_arv + v_sõnade_arv
            esimese_isiku_kasutus += esimene_isik_arv
            negatiivsus += negatiivsete_arv
            uudise_skoor = esimene_isik_arv + negatiivsete_arv
            uudise_kordaja = uudise_skoor/uudise_pikkus_sõnades
            positiivsus += positiivsete_arv
            
            pikkus.append(uudise_pikkus_sõnades)
            esimene_isik.append(esimene_isik_arv)
            neg.append(negatiivsete_arv)
            kordaja.append(round(uudise_kordaja*100, 2))
            pos.append(positiivsete_arv)

    uudisdict = {'sõnu uudises':pikkus,'esimene isik': esimene_isik, 'negatiivsus': neg, 'kordaja':kordaja, 'positiivsus':pos}

    statistika = pd.DataFrame(uudisdict, columns = ['sõnu uudises', 'esimene isik', 'negatiivsus', 'kordaja', 'positiivsus'])
    statistika['väljaanne'] = uudised['subject']  
    #statistika['pealkiri'] = uudised['title']
    statistika['link'] = uudised['link']
    
    print('Korpuses 1. isiku kasutamine', esimese_isiku_kasutus)
    print('Korpuses negatiivsus', negatiivsus)
    skoor = esimese_isiku_kasutus + negatiivsus
    print("Korpuse uudiste skoor: ", skoor)
    print("Korpuse uudiste pikkus sõnades: ", uudiste_pikkus_sõnades)
    print("Korpuse kordaja: ", round(skoor/uudiste_pikkus_sõnades*100, 2))
    print("Korpuse positiivsus", positiivsus)
    print("Korpuses negatiivseid sõnu: ", len(negatiivsed_sonad_list))#, negatiivsed_sonad_list)
    print("Korpuses positiivseid sõnu: ", len(positiivsed_sonad_list))#, positiivsed_sonad_list)
    return statistika

In [4]:
data_koos = pd.read_csv("data_uudised_koos_UUS.csv")
data_koos.shape

(17437, 5)

In [5]:
tulemused = uudise_skoor(data_koos)

  0%|                                                                                | 1/17437 [00:00<30:09,  9.64it/s]

Analüüsitakse 17437 uudist.


100%|██████████████████████████████████████████████████████████████████████████| 17437/17437 [1:26:16<00:00,  3.37it/s]


Korpuses 1. isiku kasutamine 141951
Korpuses negatiivsus 186371
Korpuse uudiste skoor:  328322
Korpuse uudiste pikkus sõnades:  7802657
Korpuse kordaja:  4.21
Korpuse positiivsus 206301
Korpuses negatiivseid sõnu:  185664
Korpuses positiivseid sõnu:  206301


In [6]:
tulemused

,sõnu uudises,esimene isik,negatiivsus,kordaja,positiivsus,väljaanne,link
0,104,0,1,0.96,0,Delfi,https://www.delfi.ee/news/paevauudised/eesti/puust-punaseks-mis-on-karantiin-ja-mis-on-eneseisolatsioon-kes-on-lahikontaktne?id=90989665
1,223,0,3,1.35,6,Telegram,https://www.telegram.ee/eesti/loomakaitseorganisatsioonid-kutsuvad-ules-boikoteerima-metsloomadega-tsirkuseid
2,322,8,12,6.21,12,Telegram,https://www.telegram.ee/eesti/nestor-alampalga-tostmine-on-valtimatu
3,459,5,5,2.18,13,Delfi,https://www.delfi.ee/news/paevauudised/eesti/eesti-uhineb-koroonavaktsiini-eelostulepinguga-millega-saab-katta-riskiruhmade-vaktsiinivajaduse?id=91697067
4,611,6,6,1.96,19,Uued Uudised,https://uueduudised.ee/arvamus/lugeja-kirjutab/tohutute-silmamoondustega-kaasaeg-vajab-seadustesse-lisaklauslit-abielu-on-mehe-ja-naise-liit/
...,...,...,...,...,...,...,...
17432,139,2,3,3.60,4,Delfi,https://www.delfi.ee/news/paevauudised/eesti/video-sopruse-puiesteel-keset-paeva-roosipoosaid-varastanud-meest-ei-tabatudki-aga-roose-enam-ei-kao?id=91495824
17433,830,29,29,6.99,13,Telegram,https://www.telegram.ee/arvamus/hamburgi-juhtiv-meditsiiniametnik-paanikaohutajatele-nakatumine-ei-tahenda-veel-haigestumist
17434,262,1,1,0.76,4,Uued Uudised,https://uueduudised.ee/uudis/eesti/rahvaloenduse-kusimustikku-lisandub-ka-randeteemaline-kusimus/
17435,470,18,9,5.74,6,Telegram,https://www.telegram.ee/maailm/liibanonlased-naeravad-euroopa-liidu-otsuse-ule-hezbollah-musta-nimekirja-kanda


In [7]:
tulemused.to_csv('meelsus_tabel_koos_UUS.csv', index = False)

In [8]:
filtr = tulemused[tulemused['sõnu uudises'] > 50].sort_values(by=['kordaja', 'positiivsus'], ascending=False)

In [9]:
filtr

,sõnu uudises,esimene isik,negatiivsus,kordaja,positiivsus,väljaanne,link
11857,114,1,27,24.56,3,Uued Uudised,https://uueduudised.ee/arvamus/repliik/delfi-retoorika-valitsusele-molli/
8808,2832,23,537,19.77,16,Telegram,https://www.telegram.ee/maailm/suur-ulevaade-usa-osalusega-sojad
13745,52,7,3,19.23,1,Eesti Päevaleht,https://epl.delfi.ee/uudised/fotod-lasnamae-keeleoskamatu-koolijuht-mulle-lihtsalt-ei-jaa-see-eesti-keel-kulge-psuhholoogiline-susteem-hakkab-vastu?id=91411233
3158,82,2,13,18.29,2,Delfi,https://www.delfi.ee/news/paevauudised/eesti/viru-vanglas-on-juba-107-nakatunut?id=91573623
9601,71,8,4,16.90,1,Telegram,https://www.telegram.ee/maailm/tony-robbinsi-intervjuu-nobeli-preemia-laureaadiga-juba-kriisi-alguses-oli-selge-et-paanikaks-pole-pohjust
...,...,...,...,...,...,...,...
16936,69,0,0,0.00,0,Delfi,https://www.delfi.ee/news/paevauudised/eesti/jurgen-ligi-nuud-on-selge-kust-tuli-kask-terviseametile?id=90163433
17216,62,0,0,0.00,0,Telegram,https://www.telegram.ee/eesti/soomes-elab-alaliselt-ligi-40-000-eesti-kodanikku
17233,55,0,0,0.00,0,Delfi,https://www.delfi.ee/news/paevauudised/valismaa/lati-nouab-edaspidi-eneseisolatsiooni-ainult-sealsest-suurema-haigestumisnaitajaga-riikidest-tulijatelt?id=91309901
17380,65,0,0,0.00,0,Uued Uudised,https://uueduudised.ee/uudis/eesti/piltuudis-ekre-malestas-sinimagede-kaitselahingus-langenuid/


In [10]:
filtr.to_csv('meelsus_tabel_koos_sorteeritud_UUS.csv', index = False)

In [11]:
#Kõrgeima kordajaga uudis
data_koos['text'][11857]

'Delfi/Eesti Päevalehe retoorika muutub päev-päevalt räigemaks. Täna avaldati Vahur Kooritsa arvamuslugu pealkirja all „Kevadine referendum annab rahvale võimaluse valitsusele molli anda“. „Rahvahääletust tagant tõukav EKRE toetub küsitlustulemustele, mille kohaselt toetab enamik Eesti inimesi abielu üksnes mehe ja naise liiduna,“ kirjutab loo autor, lisades, et küsitlus võib muutuda ametlikust teemast selliseks, „kas valijad tahavad valitsusele molli anda“. Sünonüümsõnastiku järgi tähendab “molli andma” eriti labase väljendina vastu hambaid andma, näkku lööma, peksma, kaklema. Nagu näeme, ei käi viha õhutamine meedias enam üksikisiku tasemel ja viisakate väljenditega, vaid ikka banaalselt ja grupiviisiliselt.  '

In [12]:
data_koos['text'][8808]

'Ameerika ühendriike kirjeldatakse tihti kui maailma politseid või rahuvalvajat, kuid statistika näitab hoopis teistsugust pilti: riik on siiani eksisteerinud 243 aastat ning sellest 222 aastal on see olnud mingisuguses sõjas. See tähendab, et kogu oma eluea jooksul on USA-l olnud vaid 21 aastat rahuaega, suutmata sõjata veeta järjest tervet aastakümmet. Üheks põhjuseks sellisele eksistentsile on sõjaväelis-tööstusliku kompleksi suur ning eluline roll USA majanduse elujõulisuses – riik ei suuda enam ilma sõdadeta eksisteerida.   Toome teieni ülevaatliku nimekirja kõigist USA osalusega suurematest sõdadest, kuid silmas tasub pidada, et loetelust on välja jäetud mitmed sõjaväelised sekkumised. Näiteks pole üles loetletud “rahumeelsetel meetoditel“ USA pärismaalastest puhastamine (nt petturlikud lepped), pärismaalaste hulgas korraldatud mitmed massimõrvad ning mitmed korrad, kui USA vägesid on saadetud erinevatesse riikidesse “Ameerika huve kaitsma“. Samuti on nimekirjast jäetud välja ena

In [13]:
data_koos['text'][13745]

'"Mulle lihtsalt ei jää see eesti keel külge. Ma õpin, kordan, loen tekste. Ma tean, et see pole kuigi kena, kuid ma olen koguaeg siin surve all. Mu närvisüsteem ei pea lihtsalt sellele pingele vastu ja mu psühholoogiline süsteem hakkab vastu keeleõppele."'